In [7]:
from scipy.stats import norm
import random
from math import exp, sqrt
import datetime
import numpy as np

n = norm.pdf
N = norm.cdf


def bs_price(cp_flag, S, K, T, r, v, q):
    d1 = (np.log(S / K) + (r - q + v * v / 2.) * T) / (v * np.sqrt(T))
    d2 = d1 - v * np.sqrt(T)
    if cp_flag == 'call':
        price = S * np.exp(-q * T) * N(d1) - K * np.exp(-r * T) * N(d2)
    else:
        price = K * np.exp(-r * T) * N(-d2) - S * np.exp(-q * T) * N(-d1)
    return price


def bs_vega(cp_flag, S, K, T, r, v, q):
    d1 = (np.log(S / K) + (r + v * v / 2.) * T) / (v * np.sqrt(T))
    return S * np.sqrt(T) * n(d1)


def find_vol(target_value, call_put, S, K, T, r, q):
    MAX_ITERATIONS = 1000
    PRECISION = 1.0e-5

    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_price(call_put, S, K, T, r, sigma, q)
        vega = bs_vega(call_put, S, K, T, r, sigma,q)

        diff = price - target_value

        if abs(diff) < PRECISION:
            return sigma
        sigma = sigma - diff / vega # f(x)/f'(x)
    return sigma

In [2]:
T=60/365
S=99.6
r=0.003
q=0
cp='put'
implied_vol=find_vol(125,cp,S,99.5,T,r,q)


<ipython-input-1-9b6cc837ba44>:39: RuntimeWarning: divide by zero encountered in double_scalars
  sigma = sigma - diff / vega # f(x)/f'(x)
<ipython-input-1-9b6cc837ba44>:12: RuntimeWarning: invalid value encountered in double_scalars
  d1 = (np.log(S / K) + (r - q + v * v / 2.) * T) / (v * np.sqrt(T))
<ipython-input-1-9b6cc837ba44>:22: RuntimeWarning: invalid value encountered in double_scalars
  d1 = (np.log(S / K) + (r + v * v / 2.) * T) / (v * np.sqrt(T))


In [9]:
T = 0.25
S = 3120
r = 0.0402
q = 0.036705
cp = 'call'  # call option
Strike_V_market = ((3100, 94), (3105, 90.6), (3110, 87.3), (3115, 84), (3120, 80.8),(3125, 77.7), (3130, 74.6), (3135, 71.5), (3140, 68.55))

for (a, b) in Strike_V_market:
    implied_vol = find_vol(b, cp, S, a, T, r, q)
    print('Strike: {0}\tImplied vol: {1:.2f}%'.format(a, implied_vol * 100))

Strike: 3100	Implied vol: 13.39%
Strike: 3105	Implied vol: 13.26%
Strike: 3110	Implied vol: 13.14%
Strike: 3115	Implied vol: 13.01%
Strike: 3120	Implied vol: 12.89%
Strike: 3125	Implied vol: 12.78%
Strike: 3130	Implied vol: 12.66%
Strike: 3135	Implied vol: 12.53%
Strike: 3140	Implied vol: 12.42%
